In [9]:
!pip install -q weaviate-client langchain sentence-transformers langchain_community langchain_huggingface weaviate PyPDF langchain_groq

In [10]:
!pip install -q weaviate-client==3.25.0

In [22]:
from langchain.vectorstores import Weaviate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
import weaviate
from weaviate import Client
from weaviate.auth import AuthApiKey

In [23]:
import weaviate
from weaviate.auth import AuthApiKey

url = "kankwffoqpchpczgosk3a.c0.asia-southeast1.gcp.weaviate.cloud"  # Replace with your cluster URL
api_key = "qmwoMQ4AGyGZ9ZovKLTTiq2s628ilw0bovOF"       # Replace with your Weaviate API key

client = weaviate.Client(
    url="https://kankwffoqpchpczgosk3a.c0.asia-southeast1.gcp.weaviate.cloud",
    auth_client_secret=AuthApiKey(api_key)
)
print("✅ Weaviate client ready:", client.is_ready())

✅ Weaviate client ready: True


In [24]:
loader = PyPDFLoader("law.pdf")
doc = loader.load()

In [25]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=170)
chunks = splitter.split_documents(doc)

In [26]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [27]:
vectorstore = Weaviate.from_documents(
    embedding=embedding,
    documents=chunks,  # Your documents (chunks)
    client=client  # Weaviate client
)

In [29]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"K":3})

In [30]:
from google.colab import userdata
import os
api = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"]=api

In [35]:
model = ChatGroq(model="gemma2-9b-it")

In [36]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
parser = StrOutputParser()

prompt = PromptTemplate(
    template="""You are a highly accurate assistant.
        Use ONLY the given context to answer the user's question.
        If the context does not contain the information needed, simply reply:
        "I don't know based on the given context."
        CONTEXT:
        {context}
        QUESTION:
        {question}
        Your Answer:""",
input_variables=["context", "question"])

def context(docs):
  return "\n\n".join(doc.page_content for doc in docs)

parallel_chain = RunnableParallel({
    "context":retriever | RunnableLambda(context),
    "question":RunnablePassthrough()
})


In [37]:
chain = parallel_chain | prompt | model | parser

In [38]:
query = "summary of the text"
chain.invoke(query)

'This text appears to be a table of contents for a legal document, possibly the Constitution of Pakistan. \n\nIt outlines several parts and chapters, focusing on areas like finance, property, contracts, services, armed forces, and interpretation.  The document mentions amendments and additions made to specific articles, highlighting changes over time. \n\n\n'